# 12
You have a dataset of transactions that contains the following fields:
transaction_id (integer): Unique ID for each transaction.
user_id (integer): ID of the user performing the transaction.
transaction_amount (float): Amount of the transaction.
transaction_date (string): Date of the transaction in yyyy-MM-dd format.
From this dataset, perform the following operations:
Find the top 3 users with the highest total transaction amounts.
Among these top 3 users, for each, identify the most recent transaction date.


data = [ (1, 101, 500.0, "2024-01-01"), (2, 102, 200.0, "2024-01-02"), 
(3, 101, 300.0, "2024-01-03"), (4, 103, 100.0, "2024-01-04"), 
(5, 102, 400.0, "2024-01-05"), (6, 103, 600.0, "2024-01-06"), 
(7, 101, 200.0, "2024-01-07"), ] 

columns = ["transaction_id", "user_id", "transaction_amount", "transaction_date"]

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct, avg

# Initialize Spark
spark = SparkSession.builder.getOrCreate()

# Step 1: Create DataFrame
data = [
    (1, 101, 500.0, "2024-01-01"), 
    (2, 102, 200.0, "2024-01-02"), 
    (3, 101, 300.0, "2024-01-03"), 
    (4, 103, 100.0, "2024-01-04"), 
    (5, 102, 400.0, "2024-01-05"), 
    (6, 103, 600.0, "2024-01-06"), 
    (7, 101, 200.0, "2024-01-07"),
]
columns = ["transaction_id", "user_id", "transaction_amount", "transaction_date"]

df = spark.createDataFrame(data, columns)

# Step 2: Count distinct dates per user
df_date_count = df.groupBy("user_id")\
    .agg(countDistinct("transaction_date").alias("date_count"))

# Step 3: Filter users with >= 3 transaction dates
df_filtered_users = df_date_count.filter(col("date_count") >= 3)

# Step 4: Join with original to get only relevant users
df_joined = df.join(df_filtered_users, on="user_id", how="inner")

# Step 5: Calculate average transaction amount
df_result = df_joined.groupBy("user_id")\
    .agg(avg("transaction_amount").alias("average_transaction_amount"))

df_result.display()


user_id,average_transaction_amount
101,333.3333333333333
